# 의사코드
## 변수/함수
- filename : 소스파일데이터
- find_encoding() : 인코딩 찾기 함수 
- read_file : open함수 사용 파일 읽기(읽기모드/바이너리파일)

class Veterinary :
    def 생성자 : 
        소스파일데이터
    
    def 인코딩 찾기함수(self):
        read_file    = 파일읽기
        result       = chardet라이브러리의 detect 모듈 이용 소스파일 형식 dict()형태로 반환
        charenc      = dict()형태의 인코딩 방식 추출
        open_source  = 판다스모듈 read_csv() 메서드 이용 및 인코딩 방식 이용하여 파일 읽기
        self.open_source1 = 명시적인덱서 사용 상세영업상태명 컬럼 '정상'인 문자열만 추출
        
        # return open_source1
    
    def 컬럼 선택함수(self):
        fin_source_data          = 암묵적 인덱서 사용 사용할 컬럼 추출
        fin_source_data['구분']  = 소재지 전체주소에서 제일앞의 지명만 추출하여 새 컬럼 생성
        fin_source_data.columns  = 컬럼명 재구성
        fin_source_data['도로명']컬럼 결측치 0으로 대체
        # return fin_source_data
    
    def 비교대상 리스트 생성함수(self):
        


    

In [ ]:
    def test_list(self) -> list[str]:
        self.choose_columns()
        addr_final = list()
        for i,j in self.fin_source_data[['도로명']].iterrows():
            if j[0] != 0:
                addr_final.append(j[0])
            else :
                self.fin_source_data.loc[i,"도로명"] = self.fin_source_data.loc[i,"지번"] #########
                addr_final.append(self.fin_source_data.loc[i,"지번"])
                # 비교대상 리스트
        
        return addr_final

In [ ]:
1

In [ ]:
import pandas as pd
import chardet
import re
import requests
import json
import folium
from folium.plugins import MarkerCluster
import webbrowser
from bokeh.plotting import show
from bokeh.models import ColumnDataSource, DataTable, TableColumn
import bokeh



class Veterinary :

    '''
    동물병원 전처리
    csv 출처(행안부) https://www.data.go.kr/data/15045050/fileData.do
    '''

    def __init__(self, filename :str ="./fulldata_02_03_01_P_동물병원.csv") -> None :
        self.filename = filename

    def find_encoding(self) -> pd.DataFrame:
        read_file = open(self.filename, 'rb').read()        # 파일 읽기모드 rb는 read + 바이너리 파일( 바이너리 파일 읽을때 사용 )
        result    = chardet.detect(read_file)
        charenc   = result['encoding']
        open_source = pd.read_csv(source_path, encoding=charenc)
        # self.open_source1 = open_source.loc[((open_source['상세영업상태명']=="정상") | (open_source['상세영업상태명']=="휴업")),:]
        open_source1 = open_source.loc[open_source['상세영업상태명']=="정상",:]
        print(f'파일경로 : {self.filename}')
        print(f'파일 인코딩 방식 : {charenc}')
        return open_source1

    def choose_columns(self) -> pd.DataFrame:
        open_source = self.find_encoding()
        # c = []
        # while True:
        #     print(list(self.open_source1.columns))
        #     b = input("추가할 컬럼을 선택 // 종료 0")
        #     if b == '0':
        #         break
        #     c.append(b)
        # self.df = self.open_source1.loc[:,c]
        self.fin_source_data = open_source.iloc[:,[21,10,18,19,26,27]]
        self.fin_source_data['구분'] = self.fin_source_data['소재지전체주소'].str.split(' ').str.get(0).copy()
        self.fin_source_data.columns= ['병원명', '영업상태', '지번', '도로명', '위도', '경도', '지역']
        self.fin_source_data['도로명'].fillna(0, inplace=True)
        return self.fin_source_data

    def test_list(self) -> list[str]:
        self.choose_columns()
        addr_final = list()
        for i,j in self.fin_source_data[['도로명']].iterrows():
            if j[0] != 0:
                addr_final.append(j[0])
            else :
                self.fin_source_data.loc[i,"도로명"] = self.fin_source_data.loc[i,"지번"] #########
                addr_final.append(self.fin_source_data.loc[i,"지번"])
                # 비교대상 리스트
        
        return addr_final

    def test_list2(self) -> pd.DataFrame:
        #self.test_list()
        addr_final = self.test_list()
        count = 0
        exc = list()
        for i in addr_final :
            x_y = dict(위도='y', 경도='x')
            self.rest_api_key = 'c6ead90ce93e6ff182fbb5dbbb2efb6c'
            query = i
            url = f"https://dapi.kakao.com/v2/local/search/address.json?query={query}"
            headers={'Authorization':'KakaoAK {}'.format(self.rest_api_key)}
            res = requests.get(url, headers=headers)
            res_json = res.json()
            try :
                #print(f'POINT 1{i}')
                #print(f'POINT 2{self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]}')

                if i == self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]:
                    for j,k in x_y.items():
                        self.fin_source_data.loc[ self.fin_source_data["도로명"]== i , j ] = res_json['documents'][0]['address'][k]
                elif  i == self.fin_source_data.loc[ self.fin_source_data["지번"]== i, "지번" ].unique()[0]:
                    self.fin_source_data.loc[ self.fin_source_data["지번"]== i , "도로명" ] = res_json['documents'][0]['road_address']['address_name']
                    for j,k in x_y.items():
                            self.fin_source_data.loc[ self.fin_source_data["지번"]== i , j ] = res_json['documents'][0]['address'][k]
                        
                
                #addr = res_json['documents'][0]['road_address']['address_name']
                name = self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "병원명" ].unique()[0]
                #print("point 12 comp")
                count += 1
            except IndexError:
                try:

                    i2 = i.split()
                    i2 = ' '.join(i2[:4])
                    i2 = re.sub(r'[,]','',i2)
                    query = i2
                    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={query}"
                    headers={'Authorization':'KakaoAK {}'.format(self.rest_api_key)}
                    res = requests.get(url, headers=headers)
                    res_json = res.json()

                    #print(f'POINT 3{i}')
                    #print(f'POINT 3.5{i2}')
                    #print(f'POINT 4{self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]}')

                    if i == self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]:
                        for j,k in x_y.items():
                            self.fin_source_data.loc[ self.fin_source_data["도로명"]== i , j ] = res_json['documents'][0]['address'][k]
                    elif  i == self.fin_source_data.loc[ self.fin_source_data["지번"]== i, "지번" ].unique()[0]:
                        self.fin_source_data.loc[ self.fin_source_data["지번"]== i , "도로명" ] = res_json['documents'][0]['road_address']['address_name']
                        for j,k in x_y.items():
                                self.fin_source_data.loc[ self.fin_source_data["지번"]== i , j ] = res_json['documents'][0]['address'][k]
                            
                    #addr = res_json['documents'][0]['road_address']['address_name']
                    name = self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "병원명" ].unique()[0]
                    count += 1
                except:
                    print("---- 오류 주소 내용----")
                    print(i)
                    exc.append(i)
        else:
            print(f'좌표 입력 횟수{count}')
            print(f'좌표 오류 횟수{len(exc)}')
            #print(f'좌표 오류 내용{exc}')
            return self.fin_source_data

    def make_map(self) -> folium.map:
        fin_source_data =  self.test_list2()
        m = folium.Map(location=[35.1358104681195,129.045358241731],zoom_start=8)
        marker_c = MarkerCluster().add_to(m)
        kerr = list()
        count = 0
        for _,j in fin_source_data.iterrows():
            try:
                x = j["경도"]
                y = j["위도"]
                url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={x}&y={y}"
                headers={'Authorization':'KakaoAK {}'.format(self.rest_api_key)}
                res = requests.get(url, headers=headers)
                res = res.json()
                addr=res['documents'][0]['address_name']
                x= res['documents'][0]['x']
                y= res['documents'][0]['y']
                folium.Marker(location=[y,x],tooltip=j["병원명"],icon=folium.Icon(color='red',icon='ok')).add_to(marker_c)
                
                
            except KeyError:
                print("===========================")
                print(j["병원명"])
                print("===========================")
                print(x,y)
                print("===========================")
                print(url)
                print("===========================")
                print(res)
                kerr.append(j)
                count += 1

        print (f'오류횟수{count}')
        print ("오류 목록")
        print (kerr)
        m.save('map.html')
        webbrowser.open('map.html')
            
    def make_arr(self) -> bokeh.io:
        # self.test_list2()
        fin_source_data = self.test_list2()
        source = ColumnDataSource(fin_source_data)
        columns = [ TableColumn( field=col, title=col ) for col in fin_source_data.columns ]
        data_table = DataTable(source=source, columns = columns)
        bokeh.io.save(obj=data_table, filename='table.html')
        webbrowser.open('table.html')


    def total(self) -> pd.DataFrame:
        #self.test_list2()
        self.make_map()
        self.make_arr()
        return self.fin_source_data

    def df_full(self) -> pd.DataFrame:
        pd.set_option('display.max_rows', None)
        return self.fin_source_data

        


        